In [70]:
# First I downloaded the dataset from Kaggle using the link provided in Mentormind-Resources
# I have used the help of my learnings and notes from the course, Google, Youtube, ChatGPT, etc. to make this project.

import pandas as pd
import numpy as np
import scipy.sparse
import scipy.stats
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [71]:
movies = pd.read_csv('F:\Data Science\Menternships\Movie Recommender System\Data - Content Based Filtering\movies.csv')
movies['title'] = movies['title'].str.slice(stop=-7)      #remove the Year from Title
movies.head()

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [72]:
ratings = pd.read_csv('F:/Data Science/Menternships/Movie Recommender System/Data - Content Based Filtering/ratings.csv')
ratings = ratings.drop(columns='timestamp')
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [73]:
ratings.info(), movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


(None, None)

In [74]:
df = pd.merge(movies, ratings, on='movieId', how='inner')
df

,movieId,title,genres,userId,rating
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1,4.0
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5,4.0
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7,4.5
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15,2.5
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17,4.5
...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,184,4.0
100832,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,184,3.5
100833,193585,Flint,Drama,184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,184,3.5


In [75]:
df_mean_ratings = df.groupby(['movieId','title','genres'])['rating'].mean().reset_index()

In [76]:
vectorizer = TfidfVectorizer()
movies_vec = vectorizer.fit_transform(df_mean_ratings['genres'])

In [77]:
cosine_sim = cosine_similarity(movies_vec)

In [78]:
def rec_movies(title, cosine_sim=cosine_sim, movie_ratings=df_mean_ratings):     #define the function for movie recommendation

    index = movie_ratings[movie_ratings['title']==title].index[0]                #get the index of the movie that matches the title
    
    sim_scores = list(enumerate(cosine_sim[index]))                              #get similarity scores between the movie and all movies
    
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)             #sort the movies based on similarity scores
    
    sim_scores = sim_scores[1:11]                                                #get the top 10 similar movies
    
    movie_indices = [i[0] for i in sim_scores]                                   #get the movie indices
    
    recommended_movies = movie_ratings.iloc[movie_indices][['title','rating']]   #get the movie recommendations and their ratings
    
    return recommended_movies

In [83]:
rec_movies('Jumanji')

,title,rating
53,"Indian in the Cupboard, The",3.235294
109,"NeverEnding Story III, The",2.000000
767,Escape to Witch Mountain,3.055556
1513,Darby O'Gill and the Little People,2.750000
1555,Return to Oz,3.000000
1616,"NeverEnding Story, The",3.581395
1617,"NeverEnding Story II: The Next Chapter, The",2.500000
1798,Santa Claus: The Movie,2.250000
3569,Harry Potter and the Sorcerer's Stone (a.k.a. ...,3.761682
6058,"Chronicles of Narnia: The Lion, the Witch and ...",3.443548
